In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("harishnair04/SemEval2024-Task8-SubtaskA")
dataset["train"][100]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

subtaskA_train_monolingual.jsonl:   0%|          | 0.00/347M [00:00<?, ?B/s]

subtaskA_dev_monolingual.jsonl:   0%|          | 0.00/10.8M [00:00<?, ?B/s]

subtaskA_test_monolingual.jsonl:   0%|          | 0.00/84.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/119757 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/34272 [00:00<?, ? examples/s]

{'text': "Changing the tube and tire on a motocross bike can be done with some basic tools, patience, and a bit of know-how. In this guide, we'll cover the steps to take in order to change both the front and rear tires on your bike.\n\nPart 1: Changing the Front Tire\n\n1. Set your bike on a stand. You’ll need to lift your bike up and secure it on a stand or lift before you can start working on the front tire.\n\n2. Remove the front axle cotter pin if present. Many bikes have a cotter pin that holds the front axle nut in place. If your bike has one, remove it.\n\n3. Remove the nut. Once the cotter pin is out, use an appropriate socket wrench to remove the axle nut.\n\n4. Tap the axle bolt with a rubber mallet. Tap the axle bolt lightly with a rubber mallet to loosen it from the front wheel.\n\n5. Remove front wheel cables if present. Remove any cables or wires that are connected to the front wheel, such as brake lines or speedo cables.\n\n6. Take off the wheel. With the nut and cotter 

In [ ]:
from transformers import AutoTokenizer

roberta_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large", add_prefix_space=True)


def tokenize_function(examples):
    return roberta_tokenizer(examples["text"], padding="max_length", truncation=True)


roberta_tokenized_datasets = dataset.map(tokenize_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/119757 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/34272 [00:00<?, ? examples/s]

In [ ]:
# Data collator for padding a batch of examples to the maximum length seen in the batch
from transformers import DataCollatorWithPadding
roberta_data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

In [ ]:
small_train_dataset = roberta_tokenized_datasets["train"].shuffle(seed=38).select(range(5000))
small_eval_dataset = roberta_tokenized_datasets["validation"].shuffle(seed=38)
small_test_dataset = roberta_tokenized_datasets["test"].shuffle(seed=38).select(range(5000))

In [ ]:
# Print a few samples from the training dataset
print("Training Dataset Samples:")
for i in range(5):
    print(small_train_dataset[i])

# Print a few samples from the evaluation dataset
print("\nEvaluation Dataset Samples:")
for i in range(5):
    print(small_eval_dataset[i])

# Print a few samples from the test dataset
print("\nTest Dataset Samples:")
for i in range(5):
    print(small_test_dataset[i])

Training Dataset Samples:
{'text': '\nIn this paper, we explore the mass dependence, time evolution, and degree of outflows associated with star formation in the Perseus molecular cloud. We surveyed 51 submillimetre cores and 37 of those showed evidence of protostellar outflows in their broad linewings - with 13 of those potentially being false positives due to confusion with neighbouring flows. Near-infrared detections further confirmed that 65 percent of the cores are protostars. Comparing our results to Spitzer infrared detections, outflow maps at this same sensitivity were found to be just as good as Spitzer in identifying protostars. Outflow momentum flux displays correlations with core mass and source luminosity, but with a large degree of scatter. Furthermore, our results did not provide strong evidence for the Bontemps et al. (1996) assertion of lower momentum fluxes for Class I sources in comparison to Class 0 sources. Lastly, our sample also included low momentum flux, low-lu

In [ ]:
from transformers import AutoModelForSequenceClassification

roberta_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large", num_labels=2, torch_dtype="auto")

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
roberta_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=2, lora_alpha=16, lora_dropout=0.1, bias="none",
)
roberta_model = get_peft_model(roberta_model, roberta_peft_config)
roberta_model.print_trainable_parameters()

trainable params: 1,248,258 || all params: 356,610,052 || trainable%: 0.3500


In [ ]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    # All metrics are already predefined in the HF `evaluate` package
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric= evaluate.load("f1")
    accuracy_metric = evaluate.load("accuracy")

    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    # The trainer is expecting a dictionary where the keys are the metrics names and the values are the scores.
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [ ]:
from transformers import TrainingArguments, Trainer

lr = 2e-4
batch_size = 8
num_epochs = 5

training_args = TrainingArguments(
    output_dir="RoBERTa_Large_SubtaskA_V2",
    learning_rate=lr,
    optim="adamw_torch",
    lr_scheduler_type= "cosine",
    warmup_ratio= 0.1,
    max_grad_norm= 0.3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=False,
    gradient_checkpointing=True,
    logging_steps=1,
    logging_strategy="epoch",
    run_name=f"RoBERTa_Large_SubtaskA_V2_lr_{lr}_batch_size_{batch_size}_num_epochs_{num_epochs}",
)

In [ ]:
trainer = Trainer(
    model=roberta_model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=roberta_data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: harishnair04 (harishnair04-university-of-ottawa) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
1,0.409000,0.554390,0.783963,0.805600,0.794634,0.791800
2,0.250700,0.852814,0.700058,0.958800,0.809251,0.774000
3,0.217400,0.610154,0.791349,0.870800,0.829175,0.820600
4,0.198200,0.625854,0.815842,0.811600,0.813716,0.814200
5,0.197000,0.630258,0.797816,0.847600,0.821955,0.816400


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=3125, training_loss=0.25446697265625, metrics={'train_runtime': 4612.8301, 'train_samples_per_second': 5.42, 'train_steps_per_second': 0.677, 'total_flos': 2.33941502976e+16, 'train_loss': 0.25446697265625, 'epoch': 5.0})

In [ ]:
trainer.model.save_pretrained("Roberta_AI_Content_Detection_V2")
trainer.model.push_to_hub("Roberta_AI_Content_Detection_V2", use_temp_dir=False)

adapter_model.safetensors:   0%|          | 0.00/5.01M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harishnair04/Roberta_AI_Content_Detection_V2/commit/dce9b63f18ed11ad4f381071a0ee32fd3b33b9a8', commit_message='Upload model', commit_description='', oid='dce9b63f18ed11ad4f381071a0ee32fd3b33b9a8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/harishnair04/Roberta_AI_Content_Detection_V2', endpoint='https://huggingface.co', repo_type='model', repo_id='harishnair04/Roberta_AI_Content_Detection_V2'), pr_revision=None, pr_num=None)

In [ ]:
import json

# Get predictions
predictions = trainer.predict(small_test_dataset.remove_columns("label"))
predicted_labels = np.argmax(predictions.predictions, axis=1)

# Format predictions
results = []
for i, label in enumerate(predicted_labels):
    results.append({"id": small_test_dataset[i]["id"], "label": int(label)})

# Write to JSONL
with open("Results.JSONL", "w") as f:
    for entry in results:
        json.dump(entry, f)
        f.write("\n")

print("Predictions saved to Results.JSONL")

Predictions saved to Results.JSONL


In [ ]:
!zip -r /content/RoBERTa_Large_SubtaskA_V2_Results.zip /content/Results.JSONL

	zip warning: name not matched: /content/RoBERTa_Large_SubtaskA_V1
  adding: content/Results.JSONL (deflated 87%)


In [ ]:
from google.colab import files
files.download('/content/RoBERTa_Large_SubtaskA_V2_Results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/RoBERTa_Large_SubtaskA_V2.zip /content/RoBERTa_Large_SubtaskA_V2

  adding: content/RoBERTa_Large_SubtaskA_V2/ (stored 0%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/ (stored 0%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/adapter_config.json (deflated 53%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/trainer_state.json (deflated 69%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/adapter_model.safetensors (deflated 15%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/rng_state.pth (deflated 25%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/training_args.bin (deflated 51%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/scheduler.pt (deflated 55%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/optimizer.pt (deflated 8%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-2500/README.md (deflated 66%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-1250/ (stored 0%)
  adding: content/RoBERTa_Large_SubtaskA_V2/checkpoint-1250/adapt

In [ ]:
from google.colab import files
files.download('/content/RoBERTa_Large_SubtaskA_V2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/Roberta_AI_Content_Detection_V2.zip /content/Roberta_AI_Content_Detection_V2

  adding: content/Roberta_AI_Content_Detection_V2/ (stored 0%)
  adding: content/Roberta_AI_Content_Detection_V2/adapter_config.json (deflated 53%)
  adding: content/Roberta_AI_Content_Detection_V2/adapter_model.safetensors (deflated 15%)
  adding: content/Roberta_AI_Content_Detection_V2/README.md (deflated 66%)


In [ ]:
from google.colab import files
files.download('Roberta_AI_Content_Detection_V2.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>